In [1]:
from darknet import parse_cfg, create_modules, DarkNet, get_test_input
from util import load_classes, write_results, letterbox_image, prep_image, write, display_img, cal_scaling
import torch.nn as nn
import torch
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os.path as osp

In [2]:
blocks = parse_cfg('C:/Users/Joab-PC/OneDrive - Nanyang Technological University/NTU/2020 (SEM 2)/FYP/YOLOv3/cfg/yolov3.cfg')
net_info, module_list = create_modules(blocks)
net_info

{'type': 'net',
 'batch': '64',
 'subdivisions': '16',
 'width': '416',
 'height': '416',
 'channels': '3',
 'momentum': '0.9',
 'decay': '0.0005',
 'angle': '0',
 'saturation': '1.5',
 'exposure': '1.5',
 'hue': '.1',
 'learning_rate': '0.001',
 'burn_in': '1000',
 'max_batches': '500200',
 'policy': 'steps',
 'steps': '400000,450000',
 'scales': '.1,.1'}

In [3]:
classes = load_classes("coco.names")
classes[39]

'bottle'

In [4]:
#Get path of images
root = "C:/Users/Joab-PC/OneDrive - Nanyang Technological University/NTU/2020 (SEM 2)/FYP/YOLOv3/Core/image_list"

In [5]:
imlist = [osp.join(osp.realpath('.'), root, img) for img in os.listdir(root)]
loaded_ims = [cv2.imread(x) for x in imlist]

In [6]:
inp_dim = 416
#PyTorch Variables for images
im_batches = list(map(prep_image, loaded_ims, [inp_dim for x in range(len(imlist))]))

#List containing dimensions of original images
im_dim_list = [(x.shape[1], x.shape[0]) for x in loaded_ims]
im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)

In [11]:
output

NameError: name 'output' is not defined

In [7]:
batch_size = 3
leftover = 0
if (len(im_dim_list) % batch_size):
    leftover = 1

if batch_size != 1:
    num_batches = len(imlist) // batch_size + leftover            
    im_batches = [torch.cat((im_batches[i*batch_size : min((i +  1)*batch_size,
                       len(im_batches))]))  for i in range(num_batches)]  

In [12]:
#Intialize model
model = DarkNet('C:/Users/Joab-PC/OneDrive - Nanyang Technological University/NTU/2020 (SEM 2)/FYP/YOLOv3/cfg/yolov3.cfg')
#Load weights
model.load_weights("yolov3.weights")
model.eval()
pred = model(ready_image)

output = write_results(pred, 0.3, 80)

NameError: name 'ready_image' is not defined

In [13]:
im_dim_list = torch.index_select(im_dim_list, 0, output[:,0].long())

scaling_factor = torch.min(inp_dim/im_dim_list,1)[0].view(-1,1)


output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim_list[:,0].view(-1,1))/2
output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim_list[:,1].view(-1,1))/2

NameError: name 'output' is not defined

In [ ]:
plt.imshow(resized_image)

In [ ]:
# scaling_factor = 0.691
# outputs[:,[1,3]] -= (416-scaling_factor*452) /2
# # outputs[:,[2,4]] -= (416-scaling_factor*603) /2
# outputs[:,1:5] /= scaling_factor

In [ ]:
cord1 = outputs[0][1:3]
cord1 = tuple(cord1.int())

In [ ]:
cord1

In [ ]:
cord2 = outputs[0][3:5]
cord2 = tuple(cord2.int())

In [ ]:
cord2

In [ ]:
read_image.shape

In [ ]:
test_img = cv2.rectangle(read_image, cord1, cord2, (255,0,0))
display_img(test_img)